# Segmentation fine-tuning tutorial
Tutorial from: 

https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html

For this tutorial, we will be finetuning a pre-trained Mask R-CNN model on the Penn-Fudan Database for Pedestrian Detection and Segmentation. It contains 170 images with 345 instances of pedestrians, and we will use it to illustrate how to use the new features in torchvision in order to train an object detection and instance segmentation model on a custom dataset.

In [ ]:
import os
import torch
import matplotlib.pyplot as plt

from torchvision.io import read_image
from torchvision.ops.boxes import masks_to_boxes
from torchvision import tv_tensors
from torchvision.transforms.v2 import functional as F

# Defining the Dataset
The reference scripts for training object detection, instance segmentation and person keypoint detection allows for easily supporting adding new custom datasets. The dataset should inherit from the standard `torch.utils.data.Dataset` class, and implement `__len__` and `__getitem__`.

## Writing a custom dataset for PennFudan
Let’s write a dataset for the PennFudan dataset. After downloading and extracting the zip file, the data will include the following (in addition to some others).

    PennFudanPed/
        Annotation/  # includes bounding boxes
            FudanPed00001.txt
            FudanPed00002.txt
            FudanPed00003.txt
            FudanPed00004.txt
            ...
        
        PedMasks/  # includes segmentation masks
            FudanPed00001_mask.png
            FudanPed00002_mask.png
            FudanPed00003_mask.png
            FudanPed00004_mask.png
            ...
        PNGImages/  # includes images 
            FudanPed00001.png
            FudanPed00002.png
            FudanPed00003.png
            FudanPed00004.png
            ...


Path to penn dataset:

In [ ]:
data_path = r'C:/Users/Eric/development/data/PennFudanPed/'

Dataset class just gets N masks (N binary masks, encoded as uint8, bounding boxes and their areas, labels, and images):

In [ ]:
class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.images = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))

    def __getitem__(self, idx):
        # load images and masks
        image_path = os.path.join(self.root, "PNGImages", self.images[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        image = read_image(image_path)
        mask = read_image(mask_path)
        # instances are encoded as different colors
        obj_ids = torch.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]
        num_objs = len(obj_ids)

        # split the color-encoded mask into a set of binary masks
        masks = (mask == obj_ids[:, None, None]).to(dtype=torch.uint8)

        # get bounding box coordinates for each mask
        boxes = masks_to_boxes(masks)

        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)

        image_id = idx
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        # Wrap sample and targets into torchvision tv_tensors:
        image = tv_tensors.Image(image)

        target = {}
        target["boxes"] = tv_tensors.BoundingBoxes(boxes, format="XYXY", canvas_size=F.get_size(image))
        target["masks"] = tv_tensors.Mask(masks)
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            image, target = self.transforms(image, target)

        return image, target

    def __len__(self):
        return len(self.images)

# Defining model

We will start from a pre-trained model trained on the coco dataset, and fine-tune the last layer.

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)

    return model

That’s it, this will make model be ready to be trained and evaluated on your custom dataset.

## Pulling in bits for training
There are some helper functions to simplify training and evaluating detection models. Here, we will use `engine.py` and `utils.py` from our object detection reference repo.

Just download everything under https://github.com/pytorch/vision/tree/main/references/detection to your folder and use them here:

In [ ]:
import utils
from engine import train_one_epoch, evaluate

In [ ]:
from torchvision.transforms import v2 as T

def get_transform(train):
    transforms = []
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    transforms.append(T.ToDtype(torch.float, scale=True))
    transforms.append(T.ToPureTensor())
    return T.Compose(transforms)

Following is necessary when working in windows, otherwise can set to 4. :(

In [ ]:
num_torch_workers = 0

Wrapping dataset into dataloader.
Note we haven't discussed data loader. It is discussed, in context of dataset, here:

https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
dataset = PennFudanDataset(data_path, get_transform(train=True))
data_loader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True, num_workers=num_torch_workers, collate_fn=utils.collate_fn)

Test getitem:

In [ ]:
im0, targ0 = dataset[0]

For Training we use the iterable dataloader (for more see https://stackoverflow.com/a/77654159/1886357).

Aside: Standard kinds of syntax (you wouldn't typically use `next(iter(loader))`, but would have it in a loop:

In [ ]:
images, targets = next(iter(data_loader))
type(images), type(targets)

In [ ]:
images = list(image for image in images)
# list of target dictionaries
targets = [{k: v for k, v in t.items()} for t in targets]
len(images), len(targets)

In [ ]:
# making sure __getitem__ worked
targets[0].keys()

In [ ]:
images[0].shape, images[0].permute(1, 2, 0).shape

In [ ]:
plt.imshow(images[0].permute(1,2,0));

## Getting more serious: defining functions for training

Let’s now write the main function which performs the training and the validation:

In [ ]:
# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
# our dataset has two classes only - background and person
num_classes = 2

In [ ]:
# use our dataset and defined transformations
dataset = PennFudanDataset(data_path, get_transform(train=True))
dataset_test = PennFudanDataset(data_path, get_transform(train=False))

In [ ]:
# split the dataset in train and test set (50 test sessions)
# this could be done much better
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

In [ ]:
# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=1,  #2
    shuffle=True,
    num_workers=num_torch_workers,
    collate_fn=utils.collate_fn
)

In [ ]:
data_loader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=1,
    shuffle=False,
    num_workers=num_torch_workers,
    collate_fn=utils.collate_fn
)

In [ ]:
# get the model using our helper function
model = get_model_instance_segmentation(num_classes)

In [ ]:
# move model to the right device
model.to(device);

In [ ]:
print(device)

In [ ]:
# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params,
    lr=0.005,
    momentum=0.9,
    weight_decay=0.0005)

In [ ]:
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=3,
    gamma=0.1
)

In [ ]:
# let's train it for 5 epochs just making sure things work
num_epochs = 1 #5

On the evaluation metrics:    
https://debuggercafe.com/evaluation-metrics-for-object-detection/

## Let's run training

**Note added**   
- The following doesn't seem built for practical ML eg for early stopping** -- this is what I want to change!
- Also is it weird it doesn't normalize/standardize the images like most of these faster-rcnn network tutorials do? Is that an issue?

In [ ]:
for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=20)
    # update the learning rate
    lr_scheduler.step()
    torch.cuda.empty_cache()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

    torch.cuda.empty_cache()

print("\n**DONE!**")

So after one epoch of training, we obtain a COCO-style mAP > 50, and a mask mAP of 65.  

**Note added**    
- Arguably the most important part of the fine-tuning tutorial, the above is not very helpful. No plots of learning, using different metrics for training/validation, no discussion of validation or early stopping, or practical matters  -- how am I supposed to use this tutorial with my actual data?
- My GPU is filled with stuff right now. How do I clear it?

## Inspect results
But what do the predictions look like? Let’s take one image in the dataset and verify

In [ ]:
import matplotlib.pyplot as plt
from torchvision.utils import draw_bounding_boxes, draw_segmentation_masks

In [ ]:
image = read_image("./tv_image05.png")
type(image)

In [ ]:
eval_transform = get_transform(train=False)

In [ ]:
model.eval()
with torch.no_grad():
    x = eval_transform(image)
    # convert RGBA -> RGB and move to device
    x = x[:3, ...].to(device)
    predictions = model([x, ])
    pred = predictions[0]

In [ ]:
image = (255.0 * (image - image.min()) / (image.max() - image.min())).to(torch.uint8)
image = image[:3, ...]
pred_labels = [f"pedestrian: {score:.3f}" for label, score in zip(pred["labels"], pred["scores"])]
pred_boxes = pred["boxes"].long()
output_image = draw_bounding_boxes(image, pred_boxes, pred_labels, colors="red")
masks = (pred["masks"] > 0.7).squeeze(1)
output_image = draw_segmentation_masks(output_image, masks, alpha=0.5, colors="blue")

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(output_image.permute(1, 2, 0));

The results look good!

**Note added**    
- Really? The eyeball test is our benchmark?

## Wrapping up 
In this tutorial, you have learned how to create your own training pipeline for object detection models on a custom dataset. For that, you wrote a torch.utils.data.Dataset class that returns the images and the ground truth boxes and segmentation masks. You also leveraged a Mask R-CNN model pre-trained on COCO train2017 in order to perform transfer learning on this new dataset.

For a more complete example, which includes multi-machine / multi-GPU training, check references/detection/train.py, which is present in the torchvision repository:

https://github.com/pytorch/vision/blob/main/references/detection/train.py